# Chest X-Rays

## Introduction:

In this notebook, I will walk through my thought process as I attempt to classify a data set of X-Ray images from patients (primarily children) with and without pneumonia through machine learning methods. The study from which the original dataset comes from can be found here: https://www.cell.com/cell/fulltext/S0092-8674(18)30154-5.

The goal is to determine, based only on these images, if the patient has viral, bacterial or no pneumonia.

The authors of this paper took a different approach than I to solve this problem - utilizing transfer learning and applying pre-trained models to this data. However, as I have been practicing my skills in Pytorch, I decided it would be more useful to me if I construct and train my own model for this set.


## Data:

The data set can be viewed and downloaded through Kaggle at: https://www.kaggle.com/paultimothymooney/chest-xray-pneumonia

Each file in this set is a jpeg containing an xray of an individual. Each sample either has (1) no pneumonia (2) bacterial pneumonia or (3) viral pneumonia. Here is a side by side view of a sample from each class. 

<table>
<tr>
    <td> <figure>
      <img src="https://github.com/jpsmith003/PneumoniaXrayClassification/chestxraydemo/normal.jpeg" width= "200px"/> 
      <figcaption>(1) No Pneumonia</figcaption>
    </figure></td>
    <td> <figure>
      <img src="chestxraydemo/person96_bacteria_464.jpeg"  width= "200px"/>
      <figcaption>(2) Bacterial Pneumonia</figcaption>
    </figure> </td>
    <td> <figure>
        <img src="chestxraydemo/viral.jpeg"  width= "200px"/>
      <figcaption>(3) Viral Pneumonia</figcaption>
    </figure> </td>
</tr>
</table>
    

As you can see, there are visible differences between those who are healthy and those with pneumonia. While doctors can often diagnose pneumonia through x-rays and bloodwork, I thought it might be an interesting application of machine learning and could prove useful when doctors are not available to interpret the data. Further, distinguishing between the types of pneumonia could be especially helpful since viral and bacterial pneunomia exhibit many of the same symptoms but require different treatments.

After splitting the data into different subsets for validation and testing, there are 4815 images in the training set, 482 in the validation set, and 624 in the testing set.

In [ ]:
import torch
import torchvision
import torchvision.transforms as transforms
import torchvision.datasets as datasets
import torch.nn.functional as F
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset
import os
from PIL import Image
import matplotlib.pyplot as plt
from torch.optim.lr_scheduler import StepLR
import numpy as np
from PIL import ImageFile
from torch.utils.tensorboard import SummaryWriter
from datetime import datetime

ImageFile.LOAD_TRUNCATED_IMAGES = True

torch.manual_seed(0)
torch.cuda.empty_cache()
now = datetime.now()

In [ ]:
# Device (CPU or cuda)
device = 'cuda'

# Batch size
bs = 64

# Tensorboard Writer
writer = SummaryWriter()

#Inputs
train_path = "D:/chest_xray/train/"
test_path = "D:/chest_xray/val/"

Below we utilize PyTorch's custom datasets to load in the data we have downloaded from Kaggle. In this process, we also apply a series of transformations to the images. They are as follows:

- Firstly, we resize the images to 224x224 so that they are all of equal size. 
- Secondly, we apply random transformations of translation, rotation, horizontal flips, zoom and shear. These help to diversify our training data and enable our model to account for some of the natural differences in the dataset
- Next, we ensure that the images have only 1 channel and convert the image to a tensor. The images were grayscale to begin with, this is just a safety net for future input.
- Finally, we normalize the values of the image by using the mean and standard deviation of the values in the channel for the entire set. This helps to reduce training time and improve accuracy by reducing skew in our population

In [ ]:
class XRayDataset(Dataset):

    # Read image files from directory and assign labels from file names
    def __init__(self, data_root):
        self.samples = []
        self.labels = []
        
        for infection_status in os.listdir(data_root):
            infection_folder = os.path.join(data_root, infection_status)
            
            for file in os.listdir(infection_folder):
                filepath = os.path.join(infection_folder, file)
                self.samples.append(filepath)
                
                if(infection_status == 'NORMAL'):
                    label = 0
                elif('bacteria' in file):
                    label = 1
                else:
                    label = 2
                
                self.labels.append(label)

    # Image Processing
    def transform(self, image):
        resize = transforms.Compose([
            transforms.Resize(size=(224,224)),
            transforms.RandomAffine(degrees = 4, translate=(.1,.05), shear = 0.1, scale = (0.85, 1.15)),
            transforms.RandomHorizontalFlip(),   
            transforms.Grayscale(num_output_channels=1),
            transforms.ToTensor(),
            transforms.Normalize(mean=[-0.0964],
                     std=[0.4809])
        ])
        
        return resize(image)
    
    # Required methods       
    def __len__(self):
        return len(self.samples)

    def __getitem__(self, idx):
        image = self.transform(Image.open(self.samples[idx]))
        label = self.labels[idx]
        return image, label

In [ ]:
# Loads training data set
def load_train_data():
    train_data = XRayDataset(train_path)
    train_loader = torch.utils.data.DataLoader(train_data, batch_size = bs, num_workers = 0, shuffle=True)
    return train_loader

In [ ]:
# Loads testing/validation data sets
def load_test_data():
    data = XRayDataset(test_path)
    test_loader = torch.utils.data.DataLoader(data, batch_size = bs, num_workers = 0, shuffle=True)
    return test_loader


Here's a few new samples of what images look like after the transformations:
<table>
<tr>
    <td> <figure>
      <img src="chestxraydemo/transformed (2).png"  width= "200px"/> 
        <figcaption>No Pneumonia</figcaption>
    </figure></td>
    <td> <figure>
      <img src="chestxraydemo/transformed_1.png"  width= "200px"/>
        <figcaption>Bacterial Pneumonia</figcaption>
    </figure> </td>
    <td> <figure>
        <img src="chestxraydemo/transformed (3).png"  width= "200px"/>
      <figcaption>Viral Pneumonia</figcaption>
    </figure> </td>
</tr>
</table>

In [ ]:
# # Find mean and std of the channels in the image set

# loader = load_train_data()
# mean = 0.
# std = 0.
# nb_samples = 0.
# for data,_ in loader:
#     batch_samples = data.size(0)
#     data = data.view(batch_samples, data.size(1), -1)
#     mean += data.mean(2).sum(0)
#     std += data.std(2).sum(0)
#     nb_samples += batch_samples

# mean /= nb_samples
# std /= nb_samples

## Modeling

Next, lets define our model. The process by which this model was chosen will be discussed further down in the document, however, there are a few things worth mentioning about this model.

First and foremost, the final structure is derived from the AlexNet architecture (https://papers.nips.cc/paper/4824-imagenet-classification-with-deep-convolutional-neural-networks).

It contains 5 convolutional layers with decreasing kernel size, each followed by a ReLU activation function to add some nonlinearity. Following this, we have 3 linear layers and a softmax for our final classification.

In [ ]:
class CNN(nn.Module):
    
    def __init__(self):
        super(CNN, self).__init__()           
            
        self.conv1 = nn.Conv2d(1, 64, kernel_size=11, stride=4, padding=2)
        self.conv2 = nn.Conv2d(64, 192, kernel_size=5, padding=2)
        self.conv3 = nn.Conv2d(192, 384, kernel_size=3, padding=1)
        self.conv4 = nn.Conv2d(384, 256, kernel_size=3, padding=1)
        self.conv5 = nn.Conv2d(256, 256, kernel_size=3, padding=1)
        
        self.dropout = nn.Dropout(0.5)        
        self.activation = nn.RReLU(inplace = True)
        
        self.pool = nn.AdaptiveAvgPool2d((6,6))
        
        self.dense1 = nn.Linear(256*6*6, 4096)
        self.dense2 = nn.Linear(4096, 4096)
        self.out = nn.Linear(4096, 3)
        
    def forward(self, x):

        x = self.conv1(x)
        x = self.activation(x)
        x = F.max_pool2d(x, kernel_size=3, stride = 2)
        
        x = self.conv2(x)
        x = self.activation(x)
        x = F.max_pool2d(x, kernel_size=3, stride = 2)
        
        x = self.conv3(x)
        x = self.activation(x)
        
        x = self.conv4(x)
        x = self.activation(x)
        
        x = self.conv5(x)
        x = self.activation(x)
        x = F.max_pool2d(x, kernel_size=3, stride = 2)
        
        ########
        
        x = self.pool(x)
        x = torch.flatten(x, 1)
        
        x = self.dropout(x)
        x = self.dense1(x)
        x = self.activation(x)
        

        x = self.dropout(x) 
        x = self.dense2(x)
        x = self.activation(x)
        x = F.log_softmax(x, dim=1)
        
        return x

The following functions are a relatively standard way of training and testing in Pytorch. They have only been slightly modified from the PyTorch tutorials found here: https://pytorch.org/tutorials/

In [ ]:
# Adjust weights from a single epoch
def train(optimizer, epoch):
    
    model.train()
    train_loader = load_train_data()
    
    for batch_index, (data, target) in enumerate(train_loader):
        
        # move data to device
        data, target = data.to(device), target.to(device)
        
        optimizer.zero_grad()
        output = model(data)
        loss = nn.functional.nll_loss(output, target)
        
        loss.backward()
        optimizer.step()
    
        # Display progress and write to tensorboard
        if batch_index % 10 == 0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, batch_index * len(data), len(train_loader.dataset),
                100.0 * batch_index / len(train_loader), loss.item()))
            niter = epoch*len(train_loader)+batch_index
            writer.add_scalar('Train/Loss', loss.data, niter)

In [ ]:
# Predict and score for test set
def test(epoch):
    
    model.eval()
    test_loss = 0
    correct = 0
    test_loader = load_test_data()
    results = []
    
    # Don't update model
    with torch.no_grad():
        
        # Predict
        for data, target in test_loader:
            
            data, target = data.to(device), target.to(device)
            output = model(data)
            
            test_loss += F.nll_loss(output, target, reduction='sum').item()
            pred = output.argmax(dim=1, keepdim=True)
            correct += pred.eq(target.view_as(pred)).sum().item()
            #print(pred)
    
    # Calculate accuracy        
    test_loss /= len(test_loader.dataset)
    
    # Display results
    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset),
        100.0 * correct / len(test_loader.dataset)))
    
#     return (100.0 * correct / len(test_loader.dataset))
    
    # Write to tensorboard
    writer.add_scalar('Test Accuracy', 100.0 * correct / len(test_loader.dataset), epoch)

## Model Selection

While the final model was derived from AlexNet, it was certainly not the first structure I tried.

The initial model that I trained was an attempt to get a baseline understanding of the data and ensure that the method I was planning on taking would be feasible. As such, the model consisted of only 1 convolutional layer with a ReLU activation followed by a linear layer. Here is the figure for the validation accuracy of that model as it was trained:

<table>
<tr>
    <td> <figure>
      <img src="chestxraydemo/1LayerAcc.PNG"  width= "400px"/> 
        <figcaption>Validation Accuracy of 1 Layer Model with Adam Optimizer</figcaption>
    </figure></td>
</tr>
</table>

As the accuracy increased and our training loss decreased, it was clear that there was some learnable information in the data so I added more and more layers to the model until the improvement in accuracy was no longer worth the increase in training time. The following figure was generated by a model consisting of 3 convolutional layers and 2 linear layers. Here is the validation accuracy of that model:

<table>
<tr>
    <td> <figure>
      <img src="chestxraydemo/3Layer.PNG"  width= "400px"/>
        <figcaption>Validation Accuracy of 3 Layer Model with Adam Optimizer</figcaption>
    </figure></td>
</tr>
</table>

As I tried adding more layers, the time in which the model took to train became too much to justify the marginal improvements in accuracy. Instead, I turned my focus to optimizing the performance of this model. In the above figures, I used an adam optimizer with a learning rate of 0.0001. The adam optimizer is perfect for getting a quick understanding of how the model performs (as long as the learning rate is reasonable) but often has a lower accuracy than stochastic gradient descent. Because of this, once I had found a structure I was happy with, I swapped over to SGD for my optimizer. Using a learning rate of 0.001 the previous model generated the following figures with SGD:

<table>
<tr>
    <td> <figure>
      <img src="chestxraydemo/3LayerAcc2.PNG"  width= "400px"/>
        <figcaption>Validation Accuracy of 3 Layer Model with SGD Optimizer</figcaption>
    </figure></td>
</tr>
</table>


As is visible, the models are starting to perform reasonably well. However, I wanted to try a model structure that has been proven to work well for image classification yet is still not too complicated. Hence, I turned to AlexNet - consisting of 8 total layers (5 convolutional and 3 linear), the model is not extremely complicated but has a proven track record. With an adam optimizer and a learning rate of 0.0001 the model took far fewer epochs to converge. Here is the training loss and validation accuracy:

<table>
<tr>
    <td> <figure>
      <img src="chestxraydemo/AlexNet.PNG"  width= "400px"/>
        <figcaption>Validation Accuracy of AlexNet Model with Adam Optimizer</figcaption>
    </figure></td>
    <td> <figure>
      <img src="chestxraydemo/AlexNetLoss.PNG"  width= "400px"/>
        <figcaption>Training Loss of AlexNet Model with Adam Optimizer</figcaption>
    </figure></td>
</tr>
</table>

In [ ]:
if __name__ == '__main__':
    model = CNN().to(device)
    
    #optimizer = optim.SGD(model.parameters(), lr=0.001, momentum = 0.9)
    optimizer = optim.Adam(model.parameters(), lr=0.0001)
    scheduler = StepLR(optimizer, step_size=1)

    for epoch in range(1, 1500+1):
        train(optimizer, epoch)
        test(epoch)

        # Save model
        torch.save(model.state_dict(), "D:/chest_xray/model_backup/AlexNet_3_class_xray_cnn_{}.pt".format(epoch))  
        
        
    state = {'epoch': epoch + 1, 'state_dict': model.state_dict(),
                     'optimizer': optimizer.state_dict(), 'scheduler' : scheduler}
        torch.save(state, "D:/chest_xray/model_backup/AlexNetAdamState")

In [ ]:
model = CNN().to(device)
model.load_state_dict(torch.load("D:/chest_xray/model_backup/AlexNet_3_class_xray_cnn_746.pt"))
model.eval()

test_path = "D:/chest_xray/test/"
test(1)

## Conclusions

On our test set of 624 images the AlexNet model performed reasonably well, acheiving an accuracy of 87% and producing the following confusion matrix and metrics:



<table>
<tr>
    <td> <figure>
      <img src="chestxraydemo/ConfusionMatrix.PNG"   width= "400px"/>
        <figcaption>Confusion matrix of final AlexNet model</figcaption>
    </figure></td>
        <td> <figure>
      <img src="chestxraydemo/metrics.PNG"   width= "400px"/>
        <figcaption>AlexNet Metrics</figcaption>
    </figure></td>
</tr>
</table>

Interestingly, the model is rarely wrong when predicting that a sample has pneumonia - misclassifying only 1 of this subset. An instance where this may be useful is the global shortage of testing kits for COVID-19. If a model such as this one could be applied to help diagnose patients, they would only need an xray instead (though it may just end up in xray shortages).

## Future Work

As this project is still ongoing, here are a few things I would like to try in the future:

- Coronavirus applications
- To increase accuracy, try more complicated models - will require more time to train
- Try transfer learning with pretrained models
- Wider age ranges in the data set (almost all children in this set because pneumonia is most lethal in children)
- Add more types of pneumonia
- Narrow data set to only include samples with pneumonia and then classify the type of pneumonia